## Лабораторная работа №2

In [2]:
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

from catboost import CatBoostRegressor
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm
from catboost import Pool
#import shapcc

import numpy as np
import pandas as pd 

### Предобработка

In [3]:
train = pd.read_csv('train.csv')
train.head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [4]:
train = train.dropna(subset=["target"])
train.reset_index(drop=True, inplace=True)

In [5]:
print(train.isnull().any()) # Проверяем в каких столбиках есть пропущенные значения

stock_id                   False
date_id                    False
seconds_in_bucket          False
imbalance_size              True
imbalance_buy_sell_flag    False
reference_price             True
matched_size                True
far_price                   True
near_price                  True
bid_price                   True
bid_size                   False
ask_price                   True
ask_size                   False
wap                         True
target                     False
time_id                    False
row_id                     False
dtype: bool


In [6]:
nan_columns = list(train.loc[:, train.isna().any()].columns) # Получаем названия столбцов где есть значения Nan
train[nan_columns] = train[nan_columns].fillna(train[nan_columns].median()) # Заменяем пропуски медианой
train = train[:int(len(train)*0.1)] # Берем только 10% всего датасета
train.head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,0.999883,0.999889,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,0.999883,0.999889,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,0.999883,0.999889,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,0.999883,0.999889,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,0.999883,0.999889,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [7]:
features = [col for col in train.columns if col not in ['row_id', 'time_id', 'target']] # Определяем список признаков
print(f"{len(features)} features: {features}")

14 features: ['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap']


In [8]:
X = train[features]
y = train['target']

In [9]:
number_of_days = len(X.date_id.unique()) 
number_of_days

50

In [10]:
number_of_days = len(X.date_id.unique())  # Всего 481 торговых дней, в качестве тестового набора возьмем последние n
testing_days = 10
training_days = number_of_days - testing_days
print(training_days, testing_days)

# маска, чтобы выделить дни для тренировок и тестирования
training_mask = X.date_id <= training_days
testing_mask = X.date_id > training_days

# создание обучающий и проверочный выборки
X_train, X_val, y_train, y_val = X[training_mask], X[testing_mask], y[training_mask], y[testing_mask] 
X_train.shape, X_val.shape, y_train.shape, y_val.shape

40 10


((433149, 14), (90640, 14), (433149,), (90640,))

In [11]:
def feature_engineering(X):

    _X = X.copy()

    # features taken from https://www.kaggle.com/code/zulqarnainali/explained-singel-model-optiver
    _X["liquidity_imbalance"] = _X.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    _X["matched_imbalance"] = _X.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    _X["price_spread"] = _X["ask_price"] - _X["bid_price"]
    _X['market_urgency'] = _X['price_spread'] * _X['liquidity_imbalance']
    
    return _X.drop(columns=['date_id'])

In [12]:
%%time 

# применить feature_engineering
X_train = feature_engineering(X_train)
X_val = feature_engineering(X_val)

X_train.shape, X_val.shape

CPU times: total: 15.6 ms
Wall time: 169 ms


((433149, 17), (90640, 17))

In [13]:
print(X_train.dtypes)

stock_id                     int64
seconds_in_bucket            int64
imbalance_size             float64
imbalance_buy_sell_flag      int64
reference_price            float64
matched_size               float64
far_price                  float64
near_price                 float64
bid_price                  float64
bid_size                   float64
ask_price                  float64
ask_size                   float64
wap                        float64
liquidity_imbalance        float64
matched_imbalance          float64
price_spread               float64
market_urgency             float64
dtype: object


In [14]:
# Преобразуйте все числа в формат float32, чтобы уменьшить объем памяти.
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

In [15]:
print(X_train.isnull().any()) 

stock_id                   False
seconds_in_bucket          False
imbalance_size             False
imbalance_buy_sell_flag    False
reference_price            False
matched_size               False
far_price                  False
near_price                 False
bid_price                  False
bid_size                   False
ask_price                  False
ask_size                   False
wap                        False
liquidity_imbalance        False
matched_imbalance          False
price_spread               False
market_urgency             False
dtype: bool


In [16]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True) # Заменяем бесконечности на nan
X_train = X_train.fillna(X_train.median()) # Заменяем nan на медиану

In [17]:
# Проверка на NaN и inf
assert not X_train.isnull().values.any(), "Data still contains NaNs"
assert not np.isinf(X_train.values).any(), "Data still contains infinity"

In [18]:
print(f"X train shape: {X_train.shape}")
X_train.head(5)

X train shape: (433149, 17)


,stock_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,liquidity_imbalance,matched_imbalance,price_spread,market_urgency
0,0.0,0.0,3.180603e+06,1.0,0.999812,13380277.00,0.999883,0.999889,0.999812,60651.500000,1.000026,8493.030273,1.0,0.754340,-0.615890,0.000214,0.000161
1,1.0,0.0,1.666039e+05,-1.0,0.999896,1642214.25,0.999883,0.999889,0.999896,3233.040039,1.000660,20605.089844,1.0,-0.728751,-0.815787,0.000764,-0.000557
2,2.0,0.0,3.028799e+05,-1.0,0.999561,1819368.00,0.999883,0.999889,0.999403,37956.000000,1.000298,18995.000000,1.0,0.332935,-0.714567,0.000895,0.000298
3,3.0,0.0,1.191768e+07,-1.0,1.000171,18389746.00,0.999883,0.999889,0.999999,2324.899902,1.000214,479032.406250,1.0,-0.990340,-0.213547,0.000215,-0.000213
4,4.0,0.0,4.475500e+05,-1.0,0.999532,17860614.00,0.999883,0.999889,0.999394,16485.539062,1.000016,434.100006,1.0,0.948687,-0.951109,0.000622,0.000590


In [19]:
print(f"X val shape: {X_val.shape}")
X_val.head(5)

X val shape: (90640, 17)


,stock_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,liquidity_imbalance,matched_imbalance,price_spread,market_urgency
433149,0.0,0.0,4.178100e+06,-1.0,1.000630,8.627361e+06,0.999883,0.999889,0.999065,45642.519531,1.000183,8942.000000,1.0,0.672361,-0.347450,0.001118,0.000752
433150,1.0,0.0,1.434518e+05,-1.0,0.999714,1.173337e+06,0.999883,0.999889,0.999532,17799.480469,1.000866,33006.000000,1.0,-0.299309,-0.782119,0.001334,-0.000399
433151,2.0,0.0,0.000000e+00,0.0,1.000079,3.612665e+06,0.999883,0.999889,0.999736,10501.200195,1.000079,3151.439941,1.0,0.538340,-1.000000,0.000343,0.000185
433152,3.0,0.0,1.287998e+07,-1.0,1.000552,3.989465e+07,0.999883,0.999889,0.999994,430.140015,1.000459,32275.500000,1.0,-0.973696,-0.511888,0.000465,-0.000453
433153,4.0,0.0,1.701372e+06,-1.0,1.000365,1.097762e+07,0.999883,0.999889,0.999512,15231.000000,1.000562,17534.050781,1.0,-0.070290,-0.731623,0.001050,-0.000074


# RandomForestRegressor

In [20]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import GroupKFold, cross_val_score

In [26]:
space = {
    'max_depth': hp.choice('max_depth', range(5, 15, 2)),
    'min_samples_split': hp.choice('min_samples_split', [5, 10, 20]),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 5, 1))
}

In [27]:
group_column = 'stock_id' # выбераем столбец, который лучше всего представляет группы
groups = X_train[group_column].values

In [28]:
def objective(params):
    model = RandomForestRegressor(**params)
    # groups это одномерный массив меток групп для GroupKFold
    gkf = GroupKFold(n_splits = 5)  #  количество разделений
    mae_scores = cross_val_score(model, X_train, y_train, cv = gkf, groups = groups, scoring = 'neg_mean_absolute_error')
    mae_score = np.mean(mae_scores)  # оценки отрицательные
    return {'loss': mae_score, 'status': STATUS_OK}


In [ ]:
rf_trials = Trials()
rf_best_params = fmin(fn=objective, # Функцяи для которой ищем лучшие параметры
                   space=space, # Параметры которые перебираем
                   algo=tpe.suggest, # байесовская оптимизация
                   max_evals=20, #Количество запусков
                   trials=rf_trials) # объект для сохранения истории поиска

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

In [176]:
print(f'best paarmeters: {rf_best_params}')

best paarmeters: {'max_depth': 4, 'n_estimators': 1}


In [177]:
rf_model = RandomForestRegressor(**rf_best_params)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, n_estimators=1)

In [178]:
rf_preds = rf_model.predict(X_val)
rf_mae = mean_absolute_error(y_val, rf_preds)
print(f'Mean Absolute Error: {rf_mae}')

Mean Absolute Error: 5.358939788088822


# xgboost

In [179]:
import numpy as np
import xgboost as xgb
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.ensemble import StackingRegressor


In [180]:
space = {
    'learning rate': hp.choice('learning rate', [0.01, 0.1, 0.5, 1]),
    'max_depth': hp.choice('max_depth', np.arange(5, 15, dtype=int)),
    'min_samples_split': hp.choice('min_samples_split', [5, 10, 15]),    
    #Если в результате шага разбиения дерева получается листовой узел с суммой весов экземпляров меньше min_child_weight, то процесс построения прекращает дальнейшее разбиение.
}

In [181]:
group_column = 'stock_id'
groups = X_train[group_column].values

In [182]:
def objective(params):
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['learning rate'] = int(params['learning rate'])
    
    clf = xgb.XGBRegressor(**params)
    gkf = GroupKFold(n_splits=5) 
    mae_scores = cross_val_score(clf, X_train, y_train, cv=gkf, groups=groups, scoring='neg_mean_absolute_error')
    mae_score = np.mean(mae_scores) 
    return {'loss': mae_score, 'status': STATUS_OK}


In [183]:
xgb_trials = Trials()
xgb_best_params = fmin(fn=objective,
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10, 
                   trials=xgb_trials)

100%|██████████| 10/10 [02:17<00:00, 13.80s/trial, best loss: -5.834676326496631]


In [190]:
print(f'best paarmeters: {xgb_best_params}')

best paarmeters: {'max_depth': 9, 'min_child_weight': 1}


In [186]:
xgb_best_params['max_depth'] = int(xgb_best_params['max_depth'])
xgb_best_params['min_child_weight'] = int(xgb_best_params['min_child_weight'])


xgb_model = xgb.XGBRegressor(**xgb_best_params)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [187]:
xgb_preds = xgb_model.predict(X_val)
xgb_mae = mean_absolute_error(y_val, xgb_preds)
print(f'Mean Absolute Error: {xgb_mae}')

Mean Absolute Error: 5.588524584558322


In [188]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

xgb_best_params['max_depth'] = int(xgb_best_params['max_depth'])
xgb_best_params['min_child_weight'] = int(xgb_best_params['min_child_weight'])

# Базовые модели
xgb_model = xgb.XGBRegressor(**xgb_best_params)
rf_model = RandomForestRegressor(**rf_best_params)

# Stacking
stacked_model = StackingRegressor(
    estimators=[('xgb', xgb_model), ('rf', rf_model)],
    final_estimator=LinearRegression()
)

# Обучаем модель
stacked_model.fit(X_train, y_train)

StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_ra...bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=9,
                                            max_leaves=None, min_child_weight=1,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                              ('rf',
                               RandomForestRegressor(max_depth=4,
                                                     n_estimators=1))],
                  final_estimator=LinearRegression())

In [189]:
stacked_preds = stacked_model.predict(X_val)
stacked_mae = mean_absolute_error(y_val, stacked_preds)
print(f'Mean Absolute Error: {stacked_mae}')

Mean Absolute Error: 5.346117169768797
